<a href="https://colab.research.google.com/github/zoltan23/Audio-Classification/blob/master/audio_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install python_speech_features
!pip install scipy
!pip install pandas
!pip install keras
!pip install sklearn
!pip install tqdm
!pip install librosa
!apt-get install libsndfile1 -y
!apt-get install git -y

# !rm -R Audio-Classification


!mkdir music-wav-files/clean
!mkdir music-wav-files/samples
!mkdir music-wav-files/models

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
mkdir: cannot create directory 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from scipy.io import wavfile as wav
import numpy as np

from datetime import datetime
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics

The history saving thread hit an unexpected error (OperationalError('disk I/O error',)).History will not be written to the database.


Using TensorFlow backend.


# The following code was extracted from a module that was created to train a CNN model.  It will be called to retrain the model to improve accuracy as more data becomes available from the app.  

In [ ]:
def extract_features(file_name):

    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

        mask = envelope(audio, sample_rate, 0.0005)

        mfccs = librosa.feature.mfcc(y=audio[mask], sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)

        #print("mfccsscaled", mfccsscaled.shape)
    
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
    
    return mfccsscaled

def envelope(y, rate, threshold):
    mask = []
    y = pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10), min_periods=1, center=True).mean()
    for mean in y_mean:
        if mean > threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

fulldatasetpath = '/tf/desktop/music-wav-files/wavfiles/'

metadata = pd.read_csv('/tf/desktop/music-wav-files/instruments.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():

    file_name = os.path.join(os.path.abspath(fulldatasetpath), str(row["fname"]))

    class_label = row["label"]
    data = extract_features(file_name)
    features.append([data, class_label])

    # Convert into a Panda dataframe 
    featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

    #print('Finished feature extraction from ', len(featuresdf), ' files') 


# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

# split the dataset 
x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

num_labels = yy.shape[1]
print("num_labels", num_labels)
filter_size = 2

model = Sequential()
model.add(Dense(256, input_shape=(40,)))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_labels, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

num_epochs = 15
num_batch_size = 8


start = datetime.now()

#callbacks=[checkpointer],
model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test),  verbose=1)
model.save('/tf/desktop/music-wav-files/models/model4.h5')

duration = datetime.now() - start
print("Training completed in time: ", duration)

# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])




Finished feature extraction from  1  files
Finished feature extraction from  2  files
Finished feature extraction from  3  files
Finished feature extraction from  4  files
Finished feature extraction from  5  files
Finished feature extraction from  6  files
Finished feature extraction from  7  files
Finished feature extraction from  8  files
Finished feature extraction from  9  files
Finished feature extraction from  10  files
Finished feature extraction from  11  files
Finished feature extraction from  12  files
Finished feature extraction from  13  files
Finished feature extraction from  14  files
Finished feature extraction from  15  files
Finished feature extraction from  16  files
Finished feature extraction from  17  files
Finished feature extraction from  18  files
Finished feature extraction from  19  files
Finished feature extraction from  20  files
Finished feature extraction from  21  files
Finished feature extraction from  22  files
Finished feature extraction from  23  fil

Finished feature extraction from  187  files
Finished feature extraction from  188  files
Finished feature extraction from  189  files
Finished feature extraction from  190  files
Finished feature extraction from  191  files
Finished feature extraction from  192  files
Finished feature extraction from  193  files
Finished feature extraction from  194  files
Finished feature extraction from  195  files
Finished feature extraction from  196  files
Finished feature extraction from  197  files
Finished feature extraction from  198  files
Finished feature extraction from  199  files
Finished feature extraction from  200  files
Finished feature extraction from  201  files
Finished feature extraction from  202  files
Finished feature extraction from  203  files
Finished feature extraction from  204  files
Finished feature extraction from  205  files
Finished feature extraction from  206  files
Finished feature extraction from  207  files
Finished feature extraction from  208  files
Finished f

248/248 [==============================] - 0s 1ms/step - loss: 0.3592 - accuracy: 0.8710 - val_loss: 2.6032 - val_accuracy: 0.4762
Epoch 12/15
248/248 [==============================] - 0s 838us/step - loss: 0.4363 - accuracy: 0.8589 - val_loss: 2.0315 - val_accuracy: 0.5397
Epoch 13/15
248/248 [==============================] - 0s 621us/step - loss: 0.3998 - accuracy: 0.8750 - val_loss: 1.9105 - val_accuracy: 0.6190
Epoch 14/15
248/248 [==============================] - 0s 922us/step - loss: 0.5772 - accuracy: 0.8468 - val_loss: 2.3752 - val_accuracy: 0.5556
Epoch 15/15
248/248 [==============================] - 0s 776us/step - loss: 0.5622 - accuracy: 0.7984 - val_loss: 2.2168 - val_accuracy: 0.5397


RuntimeError: Can't decrement id ref count (file write failed: time = Mon Oct 28 23:38:47 2019
, filename = '/tf/desktop/music-wav-files/models/model4.h5', file descriptor = 59, errno = 28, error message = 'No space left on device', buf = 0x7381408, total write size = 40960, bytes this sub-write = 40960, bytes actually written = 18446744073709551615, offset = 9384)

In [ ]:
def classify(filename):
    #model = load_model('/tf/desktop/music-wav-files/models/model1.h5')
    model = load_model('/tf/desktop/music-wav-files/models/model3.h5')

    def extract_feature(file_name):
   
        try:
            audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
            mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
            mfccsscaled = np.mean(mfccs.T,axis=0)
            
        except Exception as e:
            print("Error encountered while parsing file: ", file_name)
            return None, None
        print("np.array", np.array([mfccsscaled]))
        return np.array([mfccsscaled])

    #Get class labels for predicted_instrument output
    le = LabelEncoder()
    metadata = pd.read_csv('/tf/desktop/music-wav-files/instruments.csv')
    le.fit(metadata['label'])

    def print_prediction(file_name):
  
        prediction_feature = extract_feature(file_name) 

        predicted_vector = model.predict_classes(prediction_feature)
        #print("predicted_vector", predicted_vector)

        predicted_class = le.inverse_transform(predicted_vector)
        predicted_instrument = predicted_class[0]
        print("The predicted class is:", predicted_class[0], '\n') 

        predicted_proba_vector = model.predict_proba(prediction_feature) 
        predicted_proba = predicted_proba_vector[0]
        for i in range(len(predicted_proba)):
            category = le.inverse_transform(np.array([i]))
            print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )
            
        return predicted_instrument

    
    return print_prediction(filename)
    


# The following code was extracted from the unit tests we created.

In [ ]:
def predictedInstrumentTest(filename, instrument):

    predicted_instrument = classify(filename)
    print("The predicted instrument is", predicted_instrument, ". The actual instrument is", instrument, ".")
    if (instrument == predicted_instrument):
        print("Test passed!!!")
    else:
        print("Test Failed")

The history saving thread hit an unexpected error (OperationalError('disk I/O error',)).History will not be written to the database.


In [ ]:
predictedInstrumentTest('/tf/desktop/cl_C4.wav', 'Clarinet')
predictedInstrumentTest('/tf/desktop/BbClarinet.wav', 'Clarinet')
predictedInstrumentTest('/tf/desktop/trumpet4.wav', 'Trumpet')
predictedInstrumentTest('/tf/desktop/trumpet5.wav', 'Trumpet')
predictedInstrumentTest('/tf/desktop/g_sharp_real_mono.wav', 'Trumpet')
predictedInstrumentTest('/tf/desktop/trumpet_real2.wav', 'Trumpet')

NameError: name 'load_model' is not defined